# Donated to Cancer Treatment Too

* Note: As this is my first public Kernel and I'm still in learning NLP, please feel free to comment if you have any questions.
* This Kernel is modified from [the1owl: Redefining Treatment](https://www.kaggle.com/the1owl/redefining-treatment-0-57456), really thanks!
* I will highlight the main differences from the original.
* Finnally, donated to cancer treatment too.

In [1]:
from sklearn import preprocessing, pipeline, feature_extraction, decomposition, model_selection, metrics, cross_validation, svm
from sklearn.ensemble import RandomForestClassifier, AdaBoostClassifier, GradientBoostingClassifier, BaggingClassifier
from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.feature_selection import SelectKBest, chi2
from sklearn.preprocessing import normalize, Imputer
from sklearn.model_selection import GridSearchCV
from sklearn.linear_model import SGDClassifier
from sklearn.naive_bayes import MultinomialNB

import sklearn
import pandas as pd
import numpy as np
import xgboost as xgb

import datetime

C:\Users\seven\Anaconda3\lib\site-packages\sklearn\cross_validation.py:44: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)


In [2]:
train = pd.read_csv('input/training_variants')
test = pd.read_csv('input/test_variants.csv')
trainx = pd.read_csv('input/training_text', sep="\|\|", engine='python', header=None, skiprows=1, names=["ID","Text"])
testx = pd.read_csv('input/test_text.csv', sep="\|\|", engine='python', header=None, skiprows=1, names=["ID","Text"])

sol=pd.read_csv('input/stage1_solution_filtered.csv')
testx1 = pd.read_csv('input/test_text', sep="\|\|", engine='python', header=None, skiprows=1, names=["ID","Text"])
test1 = pd.read_csv('input/test_variants')

In [90]:

import numpy as np


def gini(actual, pred, cmpcol = 0, sortcol = 1):
    assert( len(actual) == len(pred) )
    all = np.asarray(np.c_[ actual, pred, np.arange(len(actual)) ], dtype=np.float)
    all = all[ np.lexsort((all[:,2], -1*all[:,1])) ]
    totalLosses = all[:,0].sum()
    giniSum = all[:,0].cumsum().sum() / totalLosses

    giniSum -= (len(actual) + 1) / 2.
    return giniSum / len(actual)

def gini_normalized(a, p):
    return gini(a, p) / gini(a, a)

    

predicted_y = np.array([0.98,0.85,0.8,0.9,0.2,0.115,0.9,0.9])
desired_y = np.array([1,1,0.1,0.1,0.1,0.1,0.9,0.9])


print (gini_normalized( desired_y,predicted_y))

0.614285714286


In [3]:
test1 = pd.merge(test1, testx1, how='left', on='ID').fillna('')
train = pd.merge(train, trainx, how='left', on='ID').fillna('')
sol["Class"]=0
for i in range(9):
    sol["Class"][sol[sol["class"+str(i+1)]==1].index]=i+1
sol=sol.drop(["class"+str(i+1) for i in range(9)],axis=1)


test1=pd.merge(test1,sol, how='left', on='ID').fillna("iss")
test1=test1.drop(test1[test1["Class"]=="iss"].index)
test1=test1.drop("ID",axis=1)
train=train.drop("ID",axis=1)
train=pd.concat([train,test1])
train["ID"]=[i for i in range(train.shape[0])]
train.head(5)

,Class,Gene,Text,Variation,ID
0,1,FAM58A,Cyclin-dependent kinases (CDKs) regulate a var...,Truncating Mutations,0
1,2,CBL,Abstract Background Non-small cell lung canc...,W802*,1
2,2,CBL,Abstract Background Non-small cell lung canc...,Q249E,2
3,3,CBL,Recent evidence has demonstrated that acquired...,N454D,3
4,4,CBL,Oncogenic mutations in the monomeric Casitas B...,L399V,4


(3689, 5)

In [4]:

y = train['Class'].values
train = train.drop(['Class'], axis=1)

test = pd.merge(test, testx, how='left', on='ID').fillna('')
pid = test['ID'].values

In [6]:
train.head(5)

,Gene,Text,Variation,ID
0,FAM58A,Cyclin-dependent kinases (CDKs) regulate a var...,Truncating Mutations,0
1,CBL,Abstract Background Non-small cell lung canc...,W802*,1
2,CBL,Abstract Background Non-small cell lung canc...,Q249E,2
3,CBL,Recent evidence has demonstrated that acquired...,N454D,3
4,CBL,Oncogenic mutations in the monomeric Casitas B...,L399V,4


### 1. Not use the codes below.
***
Not used in this Kernel.

```python
# commented for Kaggle Limits
for i in range(56):
    df_all['Gene_'+str(i)] = df_all['Gene'].map(lambda x: str(x[i]) if len(x)>i else '')
    df_all['Variation'+str(i)] = df_all['Variation'].map(lambda x: str(x[i]) if len(x)>i else '')
```

In [7]:
df_all = pd.concat((train, test), axis=0, ignore_index=True)
df_all['Gene_Share'] = df_all.apply(lambda r: sum([1 for w in r['Gene'].split(' ') if w in r['Text'].split(' ')]), axis=1)
df_all['Variation_Share'] = df_all.apply(lambda r: sum([1 for w in r['Variation'].split(' ') if w in r['Text'].split(' ')]), axis=1)

In [ ]:
df_all.head()

In [8]:
gen_var_lst = sorted(list(train.Gene.unique()) + list(train.Variation.unique()))
print(len(gen_var_lst))

3578


In [9]:
gen_var_lst = [x for x in gen_var_lst if len(x.split(' '))==1]
print(len(gen_var_lst))
i_ = 0

#commented for Kaggle Limits
for gen_var_lst_itm in gen_var_lst:
    if i_ % 100 == 0: print(i_)
    df_all['GV_'+str(gen_var_lst_itm)] = df_all['Text'].map(lambda x: str(x).count(str(gen_var_lst_itm)))
    i_ += 1

3397
0
100
200
300
400
500
600
700
800
900
1000
1100
1200
1300
1400
1500
1600
1700
1800
1900
2000
2100
2200
2300
2400
2500
2600
2700
2800
2900
3000
3100
3200
3300


In [10]:
for c in df_all.columns:
    if df_all[c].dtype == 'object':
        if c in ['Gene','Variation']:
            lbl = preprocessing.LabelEncoder()
            df_all[c+'_lbl_enc'] = lbl.fit_transform(df_all[c].values)  
            df_all[c+'_len'] = df_all[c].map(lambda x: len(str(x)))
            df_all[c+'_words'] = df_all[c].map(lambda x: len(str(x).split(' ')))
        elif c != 'Text':
            lbl = preprocessing.LabelEncoder()
            df_all[c] = lbl.fit_transform(df_all[c].values)
        if c=='Text': 
            df_all[c+'_len'] = df_all[c].map(lambda x: len(str(x)))
            df_all[c+'_words'] = df_all[c].map(lambda x: len(str(x).split(' '))) 

train = df_all.iloc[:len(train)]
test = df_all.iloc[len(train):]

In [13]:
train.head()

AttributeError: 'numpy.ndarray' object has no attribute 'head'

In [ ]:
test.head()

In [ ]:
train.shape

In [ ]:
test.shape

In [10]:
class cust_regression_vals(sklearn.base.BaseEstimator, sklearn.base.TransformerMixin):
    def fit(self, x, y=None):
        return self
    def transform(self, x):
        x = x.drop(['Gene', 'Variation','ID','Text'],axis=1).values
        return x

class cust_txt_col(sklearn.base.BaseEstimator, sklearn.base.TransformerMixin):
    def __init__(self, key):
        self.key = key
    def fit(self, x, y=None):
        return self
    def transform(self, x):
        return x[self.key].apply(str)

### 2. Main difference
***
#### 1. Pipeline Changed

The original Kernel uses the pipeline with these codes below for 'Text' feature extraction:
```python
#commented for Kaggle Limits
('pi3', pipeline.Pipeline([('Text', cust_txt_col('Text')), 
                           ('tfidf_Text', feature_extraction.text.TfidfVectorizer(ngram_range=(1, 2))), 
                           ('tsvd3', decomposition.TruncatedSVD(n_components=50, n_iter=25, random_state=12))]))
```
Unfortunately, it can not fit my memory of 8GB + 2GB(swap). And without these features, I can only get nearly 0.7xxx on PL.

So, I try to use **HashingVectorizer + TfidfTransformer** instead of **TfidfVectorizer**. [Reference](http://scikit-learn.org/stable/modules/feature_extraction.html#vectorizing-a-large-text-corpus-with-the-hashing-trick)

#### 2. Parameter Tuning

For HashingVectorizer saved my memory, I try to use **ngram_range=(1, 3)** with HashingVectorizer. 

And **n_components=300** with **TruncatedSVD**.

#### 3. Batch Transform

With these codes, I can fit_transform the **train**, but still out of memory if transform **test**.

So, I try to use batch transform of test data step by step, and vstack all. 

And, it works!

In [11]:
import function as fun
print('Pipeline...')
fp = pipeline.Pipeline([
    ('union', pipeline.FeatureUnion(
        n_jobs = 3,
        transformer_list = [
            ('standard', fun.cust_regression_vals()),
            ('pi1', pipeline.Pipeline([('Gene', fun.cust_txt_col('Gene')), 
                                       ('count_Gene', feature_extraction.text.CountVectorizer(analyzer=u'char', ngram_range=(1, 8))), 
                                       ('tsvd1', decomposition.TruncatedSVD(n_components=20, n_iter=25, random_state=12))])),
            ('pi2', pipeline.Pipeline([('Variation', fun.cust_txt_col('Variation')), 
                                       ('count_Variation', feature_extraction.text.CountVectorizer(analyzer=u'char', ngram_range=(1, 8))), 
                                       ('tsvd2', decomposition.TruncatedSVD(n_components=20, n_iter=25, random_state=12))])),
            #commented for Kaggle Limits
             ('pi3', pipeline.Pipeline([('Text', fun.cust_txt_col('Text')), 
                                        ('hv', feature_extraction.text.HashingVectorizer(decode_error='ignore', n_features=2 ** 16, non_negative=True, ngram_range=(1, 3))),
                                        ('tfidf_Text', feature_extraction.text.TfidfTransformer()), 
                                        ('tsvd3', decomposition.TruncatedSVD(n_components=300, n_iter=25, random_state=12))]))

        
        ])
    )])



Pipeline...


In [12]:
print("done")
train = fp.fit_transform(train)
print (train.shape)

test_t = np.empty([0, train.shape[1]])
step = 200
for i in range(0, len(test), step):
    step_end = i+step
    step_end = step_end if step_end < len(test) else len(test)
    _test = fp.transform(test.iloc[i:step_end])
    test_t = np.vstack((test_t, _test))
test = test_t
print (test.shape)

done
(3689, 3747)
(986, 3747)


### 3. Xgboost Parameter Tuning
***
#### 1. eta 0.03333 -> 0.02 

I like small learning rate.

#### 2. max_depth 4 -> 6 

Bigger means higher risk of overfitting. But, since we got more features, maybe it could be better for this big data?! I'm not sure yet.

#### 3. test_size 0.18 -> 0.15

With slightly more data to train.

In [14]:
y = y - 1 #fix for zero bound array

In [15]:

file_pre = datetime.datetime.now().strftime('%m_%d_%H_%M_%S')

denom = 0
fold = 30 #Change to , 1 for Kaggle Limits
for i in range(fold):
    params = {
        #'eta': 0.03333,
        'eta': 0.02,
        #'max_depth': 4,
        'max_depth': 6,
        'objective': 'multi:softprob',
        'eval_metric': 'mlogloss',
        'num_class': 9,
        'seed': i,
        'silent': True
    }
    x1, x2, y1, y2 = model_selection.train_test_split(train, y, test_size=0.15, random_state=i)
    watchlist = [(xgb.DMatrix(x1, y1), 'train'), (xgb.DMatrix(x2, y2), 'valid')]
    model = xgb.train(params, xgb.DMatrix(x1, y1), 1000,  watchlist, verbose_eval=50, early_stopping_rounds=100)
    score1 = metrics.log_loss(y2, model.predict(xgb.DMatrix(x2), ntree_limit=model.best_ntree_limit), labels = list(range(9)))
    print(score1)
    #if score < 0.9:
    if denom != 0:
        pred = model.predict(xgb.DMatrix(test), ntree_limit=model.best_ntree_limit+80)
        preds += pred
    else:
        pred = model.predict(xgb.DMatrix(test), ntree_limit=model.best_ntree_limit+80)
        preds = pred.copy()
    denom += 1
    submission = pd.DataFrame(pred, columns=['class'+str(c+1) for c in range(9)])
    submission['ID'] = pid
    submission.to_csv('submission_xgb_fold_'  + str(i) + '_' + file_pre + '.csv', index=False)
preds /= denom
submission = pd.DataFrame(preds, columns=['class'+str(c+1) for c in range(9)])
submission['ID'] = pid
submission.to_csv('submission_xgb_' + file_pre + '.csv', index=False)

[0]	train-mlogloss:2.15584	valid-mlogloss:2.16359
Multiple eval metrics have been passed: 'valid-mlogloss' will be used for early stopping.

Will train until valid-mlogloss hasn't improved in 100 rounds.
[50]	train-mlogloss:1.15495	valid-mlogloss:1.38552
[100]	train-mlogloss:0.776327	valid-mlogloss:1.12099
[150]	train-mlogloss:0.568953	valid-mlogloss:1.00452
[200]	train-mlogloss:0.44558	valid-mlogloss:0.950147
[250]	train-mlogloss:0.354375	valid-mlogloss:0.923852
[300]	train-mlogloss:0.283331	valid-mlogloss:0.911602
[350]	train-mlogloss:0.229348	valid-mlogloss:0.907653
[400]	train-mlogloss:0.185288	valid-mlogloss:0.909563
[450]	train-mlogloss:0.151876	valid-mlogloss:0.917215
Stopping. Best iteration:
[355]	train-mlogloss:0.224587	valid-mlogloss:0.907363



ValueError: The type of target data is not known

In [18]:
xgb.DMatrix(x2)

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline

plt.rcParams['figure.figsize'] = (7.0, 7.0)
xgb.plot_importance(booster=model,); plt.show()

### References
* [Redefining Treatment](https://www.kaggle.com/the1owl/redefining-treatment-0-57456)
* [Vectorizing a large text corpus with the hashing trick](http://scikit-learn.org/stable/modules/feature_extraction.html#vectorizing-a-large-text-corpus-with-the-hashing-trick)
* [HashingVectorizer](http://scikit-learn.org/stable/modules/generated/sklearn.feature_extraction.text.HashingVectorizer.html)